# Tutorial - Binary Classification with Spark and Scala
This notebook shows how you can perform [binary classification](http://en.wikipedia.org/wiki/Binary_classification) using IBM Knowledge Anyhow Workbench and Apache Spark. In particular, this notebook shows how you can solve a common machine learning problem: spam classification. This notebook uses a small dataset to highlight the usage of Spark but it can scale to use big data without changing any code.

This notebook will cover the following steps:

1. Connect to a Spark cluster
2. Load training and test datasets
3. Train the machine learning model
4. Test the machine learning model

## Background
***Logistic Regression***

A [Logistic Regression model](http://en.wikipedia.org/wiki/Logistic_regression) is used to classify items into one of two possible categories (binary classification). The model is a linear separating plane which is used to identify the classification of items in the set. All items on one side of the linear separating plane belong to one category and all items on the other side of the linear separating plane belong to the other category.

There are several ways to train a Logistic Regression model. This notebook uses a [Stochastic Gradient Descent (SGD)](http://en.wikipedia.org/wiki/Stochastic_gradient_descent) implementation because [Spark MLlib](https://spark.apache.org/docs/latest/mllib-guide.html) includes an implementation of SGD in its library. SGD is an appropriate implementation for large-scale machine learning problems so, while this notebook does not use a big dataset, it's an appropriate example for Spark.


***Dataset***

This notebook uses the [Spambase dataset](https://archive.ics.uci.edu/ml/datasets/Spambase) which was created by Mark Hopkins, Erik Reeber, George Forman, and Jaap Suermondt at HP Labs. It includes 4601 observations corresponding to email messages, 1813 of which are spam. The spambase data has 57 real valued explanatory variables which characterize the contents of an email and one binary response variable indicating if the email is spam. Of the 57 explanatory variables, 48 describe word frequency, 6 describe character frequency, and 3 describe sequences of capital letters. 
The dataset may be found [here](https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/).

##Connect to a Spark Cluster

First, instruct a provisioner to create a Spark Kernel on the Spark cluster.

In [0]:
nitro.spark.create_kernel("http://169.53.153.8:9000/")

When the code above reports "Kernel ready", your notebook is now connected to a Scala kernel on your Spark cluster. Commands you issue from here on out execute in that kernel. Now, tell the kernel to import the required libraries.

In [1]:
import org.apache.spark.SparkContext
import org.apache.spark.mllib.classification.LogisticRegressionWithSGD
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.util.MLUtils

## Load training and test datasets

The dataset is stored on the Spark cluster. Load the dataset from HDFS on the Spark cluster.

In [2]:
val data = sc.textFile("hdfs://10.122.48.12:8020/data/email-data.csv")

The dataset includes 4,601 records which have already been [featurized](http://en.wikipedia.org/wiki/Feature_%28machine_learning%29). Each record contains 57 feature scores (characteristics of the email) followed by a classification of the email (spam or not spam).

In [3]:
println("Total records in dataset: " + data.count())

Total records in dataset: 4601


In [5]:
// record format: <feature_1>;<feature_2>;<feature_3>;...<feature_57>;<classification>
println("Sample record:")
data.take(1)

Sample record:


Array(0;0.64;0.64;0;0.32;0;0;0;0;0;0;0.64;0;0;0;0.32;0;1.29;1.93;0;0.96;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0.778;0;0;3.756;61;278;1)

***Transform the dataset***

Spark MLlib provides a machine learning API which can work with data represeted in [LabeledPoints](https://spark.apache.org/docs/0.8.1/api/mllib/org/apache/spark/mllib/regression/LabeledPoint.html). `LabeledPoint` represents data in the following format:

```<classification-label>,<vector-of-features>```

Transform the dataset using LabeledPoints.

In [6]:
// LabeledPoint constructor expects (<label>,<feature_vector>) so parse appropriately
val parsedData = data.map { line =>
  val parts = line.split(';')
  // Label is in the 57th position (0-indexed). Everything but the right-most item is a feature 
  LabeledPoint(parts(57).toDouble, Vectors.dense(parts.dropRight(1).map(_.toDouble)))
}.cache()

Each transformed record now looks like this:

```<label>,[<feature_1>,<feature_2>,<feature_3>,...<feature_57>]```

In [8]:
println("Sample of record transformed using LabeledPoint:")
parsedData.take(1)

Sample of record transformed using LabeledPoint:


Array((1.0,[0.0,0.64,0.64,0.0,0.32,0.0,0.0,0.0,0.0,0.0,0.0,0.64,0.0,0.0,0.0,0.32,0.0,1.29,1.93,0.0,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.778,0.0,0.0,3.756,61.0,278.0]))

In [9]:
println("Total records in transformed dataset: " + parsedData.count())

Total records in transformed dataset: 4601


Machine learning models rely on training sets from which the function for calculating the linear separating plane is derived. Additonally, machine learning models rely on test sets which can be used to assess the performance of the model. Generate the training and test sets from the original dataset.

In [10]:
// Split data into training (80%) and test (20%).
val splits = parsedData.randomSplit(Array(0.8, 0.2), seed = 11L)
val training = splits(0).cache()
val test = splits(1)

In [11]:
println("Total records in training set: " + training.count())
println("Total records in test set: " + test.count())

Total records in training set: 3616
Total records in test set: 985


## Train the machine learning model

The Spark cluster is a shared environment. The training of the model (which you'll run in the next code cell) is the most resource-intensive step in this notebook. You may experience a delay in the completion of the training step. If the training step does not complete within a reasonable time (<10 mins) then you should try to restart the kernel.

In [13]:
// Train the model using the MLlib API
val numIterations = 500
val model = LogisticRegressionWithSGD.train(training, numIterations)

## Test the machine learning model

At this point, you have a Logistic Regression model trained to classify featurized emails as spam or not spam. It is important to test the model using a set of emails with known classifications.

Use the model's predict function to test it against the test set which you created above. Store the predicted classification of each test record in a list that maps to the known classification of each test record so we can easily compute the performance of the model.

In [24]:
// Create an RDD of predicted and known classifications for the test set.
val predictedAndKnown = test.map { record =>
  val predicted = model.predict(record.features)
  (predicted, record.label)
}

Create an array from the [RDD](http://spark.apache.org/docs/1.2.1/api/scala/index.html#org.apache.spark.rdd.RDD) for easy iteration.

In [25]:
var predictedAndKnown_arr = predictedAndKnown.collect()

Print the first 5 predicted and known classifications.

In [26]:
for( i <- 0 to 4 ){
   println(predictedAndKnown_arr(i))
}

(1.0,1.0)
(1.0,1.0)
(0.0,1.0)
(1.0,1.0)
(1.0,1.0)


Iterate through the array to compare the predicted classification against the known classification. Keep a count of the correct classifications to determine how well the model performs.

In [28]:
var correctCount = 0
predictedAndKnown_arr.foreach{i =>
    if(i._1 == i._2){ //If this is true then model predicted correctly
      correctCount = correctCount + 1;
    }
}

In [29]:
println("Correct predictions: " + correctCount)

Correct predictions: 695


The [quality of a model](http://en.wikipedia.org/wiki/Precision_and_recall) is often measured by accuracy. Accuracy is the percentage of correct predictions made by a model (correct predictions / total predictions). Calculate the accuracy of your model.

In [30]:
println("Model Accuracy: " + correctCount.toFloat/test.count().toFloat)

Model Accuracy: 0.70558375


## Conclusion
In this notebook, you created a binary classification model using a Spark cluster. You used core Spark functionality and the Spark MLlib library to:
* load and transform pre-existing data
* train a machine learning model
* test a machine learning model

## References
1. [Machine Learning Repository: Spambase Dataset](https://archive.ics.uci.edu/ml/datasets/Spambase) 
2. [Learning Spark, Lightning Fast Data Analysis](http://shop.oreilly.com/product/0636920028512.do) by Holden Karau, Andy Konwinski, Patrick Wendell, Matei Zaharia; O'Reilly Media
3. [Spark MLlib Logistic Regression Doc](https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression)

***Additional Spark/MLlib Resources***
* [Main Page](https://spark.apache.org/docs/latest/)
* [Programming Guide](https://spark.apache.org/docs/latest/programming-guide.html)
* [MlLib](https://spark.apache.org/docs/latest/mllib-guide.html)
* [Logistic Regression](https://spark.apache.org/docs/latest/mllib-linear-methods.html#logistic-regression)